In [17]:
import pandas as pd
from web3 import Web3
import yaml
import seaborn as sns

## Conecting to local node

In [15]:
with open(r'provider_key.yaml') as file:
    provider_params = yaml.load(file, Loader=yaml.FullLoader)
alchemy = provider_params['key']
w3 = Web3(Web3.HTTPProvider(alchemy))

w3.isConnected()

True

## Loading data

In [3]:
data = pd.read_json('probability.json')
data.head()

,lootId,score,rarest
0,1091542,40.277757,1
1,884118,39.427093,2
2,435990,39.409585,3
3,549134,37.910700,4
4,868454,37.862941,5


# Finding opportunities

This is a script to find the first rare more_loot pice that is still available

### **Checking all unclaimed rarest bags**

In [ ]:
address = '0x1dfe7Ca09e99d10835Bf73044a23B73Fc20623DF'
with open('more_loot_abi.json', 'r') as file:
    abi = file.read()

mloot = w3.eth.contract(address=address, abi=abi)
mloot.functions.totalSupply().call()

#### **Execution**

In [ ]:
data.loc[:, 'current_owner'] = ''
curr_owners = []
for index, row in data.iterrows():
    try:
        owner = mloot.functions.ownerOf(row['lootId']).call()
    except Exception as e:
        print(f" #### Just flagged loot:{row['lootId']}, might be a candidate ### /n")
        owner = ''
    
    if owner != '':
        curr_owners.append(owner)
        #print(f"..tried loot: {row['lootId']}, but there is an owner already: {owner}")
    else:
        print(f" #### Try:{row['lootId']}, score:{row['score']}, rarest order:{row['rarest']} !!### /n")
        curr_owners.append('possible candidate')

### **Scraping some recent offers on open sean to spot rare bags being sold for cheap**

#### **Creating the functions to get and check the data**

In [4]:
max_target = 1387

selected_ids = data.loc[data.rarest<=max_target, :]
selected_ids_formated = [str(id) for id in selected_ids.loc[:, 'lootId'].values]

In [5]:
import requests
import json
from decimal import Decimal

def make_query(ids:list, offset_:str="0"):
    #https://docs.opensea.io/reference/retrieving-orders
    
    url = "https://api.opensea.io/wyvern/v1/orders"
    querystring = {"asset_contract_address":"0x1dfe7Ca09e99d10835Bf73044a23B73Fc20623DF",
               "token_ids": ids,
               "sale_kind": 0,
               "side":1,
               "is_english": "false",
               "bundled":"false",
               "include_bundled":"false",
               "include_invalid":"false",
               "limit":"50",
               "offset":offset_,
               "order_by":"created_date",
               "order_direction":"desc"}

    headers = {"Accept": "application/json"}

    response = requests.request("GET", url, headers=headers, params=querystring)
    try:
        formated_response = json.loads(response.text)['orders']
    except KeyError:
        formated_response = []
    
    return formated_response

def extract_order(order_list:list):
    orders_final = []
    for order in order_list:
        orders_final.append({
            "lootId": int(order["asset"]["token_id"]),
            "listing_create_date": order["created_date"],
            "price_ETH": float((Decimal(order["current_price"]) / 10 ** 18).to_eng_string())
        })
    return orders_final
        

def check_candidates_offers(candidates_list:list):
    candidates_number = len(candidates_list)
    number_of_loops = candidates_number / 30 # 30 max length for OpenSea  token_ids[] 
   
    checked_candidates = []
    
    candidates_count = 0
    while candidates_count <= candidates_number:
        candidates_upper_band = candidates_count + 30
        data = make_query(ids=candidates_list[candidates_count: candidates_upper_band], offset_="0")
        
        if (data != []):
            if (len(data)==50):
                extracted_data = extract_order(data)
                checked_candidates += extracted_data
                
                offset_counter = 50
                length_new_query = 1
                while length_new_query > 0:
                    new_query = make_query(ids=candidates_list[candidates_count: candidates_upper_band], offset_=str(offset_counter))
                    if (new_query == []):
                        length_new_query = 0
                    else:
                        extracted_new_query = extract_order(new_query)
                        checked_candidates += extracted_new_query
                        offset_counter += 50
            else:
                extracted_data = extract_order(data)
                checked_candidates += extracted_data
        
        candidates_count += 30
    
    return checked_candidates
                
                
#make_query(ids=selected_ids_formated[:30], offset_="0")

#### **Playing with the results**

In [6]:
checked_ids = check_candidates_offers(selected_ids_formated)
checked_ids_dt = pd.DataFrame.from_records(checked_ids)

selected_ids = selected_ids.merge(checked_ids_dt, on='lootId', how='left')
listed_rare_ids = selected_ids.loc[selected_ids.price_ETH.notnull()]
listed_rare_ids.head()

,lootId,score,rarest,listing_create_date,price_ETH
18,1302577,34.566702,19,2021-09-04T19:42:56.734914,2.0
31,458638,32.395416,32,2021-09-04T20:47:27.440201,5.0
53,100603,31.166084,54,2021-09-04T18:01:51.786994,1.0
83,11023,29.772761,84,2021-09-05T07:40:16.311218,1.0
84,11023,29.772761,84,2021-09-04T16:31:03.717083,100.0


Finding cheap opportunities

In [8]:
listed_rare_ids.sort_values(['rarest', 'price_ETH'], ascending=[True, True]).head(10)

,lootId,score,rarest,listing_create_date,price_ETH
18,1302577,34.566702,19,2021-09-04T19:42:56.734914,2.000
31,458638,32.395416,32,2021-09-04T20:47:27.440201,5.000
53,100603,31.166084,54,2021-09-04T18:01:51.786994,1.000
83,11023,29.772761,84,2021-09-05T07:40:16.311218,1.000
84,11023,29.772761,84,2021-09-04T16:31:03.717083,100.000
192,592440,27.780361,192,2021-09-05T13:55:21.416052,0.499
193,592440,27.780361,192,2021-09-05T02:19:00.106353,0.500
253,739310,27.158157,252,2021-09-05T23:48:50.427898,100.000
257,174451,27.100473,256,2021-09-05T23:48:31.589317,125.000
259,1239622,27.097005,258,2021-09-04T18:51:27.700334,50.000


In [10]:
listed_rare_ids.sort_values(['rarest', 'price_ETH'], ascending=[False, True]).head(10)

,lootId,score,rarest,listing_create_date,price_ETH
431,289394,25.455407,430,2021-09-04T19:05:27.120251,0.20
417,458499,25.600583,416,2021-09-04T19:49:24.722512,25.00
416,623873,25.603965,415,2021-09-04T21:04:45.127918,0.75
414,592469,25.625570,413,2021-09-04T22:06:20.392662,5.00
402,293811,25.701544,401,2021-09-04T22:06:30.964670,10.00
401,156394,25.711497,400,2021-09-04T22:08:38.107081,15.00
394,927514,25.778566,393,2021-09-05T23:47:26.045386,100.00
365,471213,25.952189,364,2021-09-05T13:36:51.628491,2.00
343,921184,26.231792,342,2021-09-05T23:48:05.902167,100.00
289,1096792,26.782921,288,2021-09-05T23:49:12.611890,100.00


In [9]:
sns.

26

https://eth-mainnet.alchemyapi.io/v2/Sq3Q3A7autnGe9CAYoAKysaGmE_YslMQ


### **Getting all listings for a collection..**

#### **Need to work on this yet.. not sure if it's worth once I have the above method already working..**

In [ ]:
# import requests

# def make_asset_query(offset_:str="0"):
#     url = "https://api.opensea.io/api/v1/assets"
#     querystring = {"asset_contract_address":"0x1dfe7Ca09e99d10835Bf73044a23B73Fc20623DF",
#                    "order_by": "pk",
#                    "order_direction":"desc",
#                    "offset":offset_,
#                    "limit":"50"}

#     headers = {"Accept": "application/json"}

#     response = requests.request("GET", url, headers=headers, params=querystring)
#     print(response.text)
#     return json.loads(response.text)